In [1]:
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

1.10.0


In [2]:
CSV_COLUMNS = ['radius', 'height', 'volume']
LABEL_COLUMN = 'volume'
DEFAULTS = [[0.0], [0.0], [0.0]]

def read_dataset(filename, mode, batch_size = 512):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.decode_csv(value_column, record_defaults= DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL_COLUMN)
            return features, label if mode != tf.estimator.ModeKeys.PREDICT else features
        
        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        filenames_dataset = tf.data.Dataset.list_files(filename)
        # Read lines from text files
        textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = textlines_dataset.map(decode_csv)
        
        if mode == tf.estimator.ModeKeys.TRAIN:        # if training
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:        # if evaluating or validating, epochs = 1 since it doesn't have to repeat
            num_epochs = 1 # end-of-input after this
        
        # make the dataset repeat as the number of epochs, and then batch them to specific batch size
        dataset = dataset.repeat(num_epochs).batch(batch_size)
    
        return dataset.make_one_shot_iterator().get_next()        # # Creates an Iterator for enumerating the elements of this dataset
    return _input_fn

In [3]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('radius'),
    tf.feature_column.numeric_column('height'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

In [4]:
# Serving Input Function
# Defines the expected shape of the JSON feed that the model
# will receive once deployed behind a REST API in production.
def serving_input_fn():
  feature_placeholders = {
    'radius' : tf.placeholder(tf.float32, shape=[None]),
    'height' : tf.placeholder(tf.float32, shape=[None]),
  }
  # You can transforma data here from the input format to the format expected by your model.
  features = feature_placeholders # no transformation needed
  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)  # input_fn for training and for commercial use

In [5]:
def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.DNNRegressor(
        model_dir=output_dir,
        feature_columns=feature_cols,
        hidden_units=[32,16,8],
        optimizer=tf.train.AdamOptimizer(
            learning_rate=0.1,
        )
    )

    # train specs are parameters to run training
    train_spec=tf.estimator.TrainSpec(
                       input_fn = read_dataset('./data/cylinder_train*.csv', mode = tf.estimator.ModeKeys.TRAIN),
                       max_steps = num_train_steps)        # use max_steps here since epochs is infinite
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)

    # eval specs are parameters to run evaluation
    eval_spec=tf.estimator.EvalSpec(
                       input_fn = read_dataset('./data/cylinder_valid*.csv', mode = tf.estimator.ModeKeys.EVAL),
                       steps = None,        # no stepping because evaluation is for testing accuracy, not training
                       start_delay_secs = 1, # start evaluating after N seconds
                       throttle_secs = 1,  # evaluate every N seconds
                       exporters = exporter)        # eval is designed to be realistic, so exporter instead of training dataset will be used here
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    
    return estimator

In [6]:
OUTDIR = './ouput_cylinder_dnn'

In [9]:
# load model
estimator = tf.estimator.DNNRegressor(
    model_dir=OUTDIR,
    feature_columns=feature_cols,
    hidden_units=[32,16,8],
    optimizer=tf.train.AdamOptimizer(
        learning_rate=0.1,
    )
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './ouput_cylinder_dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f16385c3198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
def make_predict_input_fn(r, h):
    def predict_input_fn():
        """An input function for prediction"""
        # Convert the inputs to a Dataset.
        dataset = tf.data.Dataset.from_tensor_slices({
                'radius' : [r],
                'height' : [h],
            })
        dataset = dataset.batch(1)
        # Return the dataset.
        return dataset.make_one_shot_iterator().get_next()
    return predict_input_fn

In [14]:
# predict according to input
r = float(input('radius: '))
h = float(input('height: '))

predictions = estimator.predict(input_fn=make_predict_input_fn(r,h))
print(next(predictions))

radius: 0.5
height: 0.8
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ouput_cylinder_dnn/model.ckpt-80000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([1.2239531], dtype=float32)}


In [8]:
# Run training
# only run this when you want to re-train
# shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
estimator = train_and_evaluate(OUTDIR, num_train_steps = 80000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './ouput_cylinder_dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f163885dba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Callin

INFO:tensorflow:global_step/sec: 103
INFO:tensorflow:loss = 4356.2627, step = 7101 (0.971 sec)
INFO:tensorflow:global_step/sec: 102.299
INFO:tensorflow:loss = 8605.578, step = 7201 (0.978 sec)
INFO:tensorflow:global_step/sec: 98.9894
INFO:tensorflow:loss = 4525.6855, step = 7301 (1.010 sec)
INFO:tensorflow:global_step/sec: 98.9108
INFO:tensorflow:loss = 4632.5166, step = 7401 (1.011 sec)
INFO:tensorflow:global_step/sec: 96.2057
INFO:tensorflow:loss = 3843.981, step = 7501 (1.040 sec)
INFO:tensorflow:global_step/sec: 100.514
INFO:tensorflow:loss = 6338.213, step = 7601 (0.995 sec)
INFO:tensorflow:global_step/sec: 98.2708
INFO:tensorflow:loss = 4406.623, step = 7701 (1.018 sec)
INFO:tensorflow:global_step/sec: 101.258
INFO:tensorflow:loss = 5262.8696, step = 7801 (0.988 sec)
INFO:tensorflow:global_step/sec: 98.5868
INFO:tensorflow:loss = 9258.543, step = 7901 (1.014 sec)
INFO:tensorflow:global_step/sec: 100.947
INFO:tensorflow:loss = 4253.378, step = 8001 (0.991 sec)
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 93.2226
INFO:tensorflow:loss = 2796.4058, step = 15401 (1.073 sec)
INFO:tensorflow:global_step/sec: 102.669
INFO:tensorflow:loss = 1123.8771, step = 15501 (0.974 sec)
INFO:tensorflow:global_step/sec: 99.333
INFO:tensorflow:loss = 1144.7366, step = 15601 (1.007 sec)
INFO:tensorflow:global_step/sec: 95.3272
INFO:tensorflow:loss = 3169.5999, step = 15701 (1.049 sec)
INFO:tensorflow:global_step/sec: 91.0838
INFO:tensorflow:loss = 1152.855, step = 15801 (1.098 sec)
INFO:tensorflow:global_step/sec: 95.2643
INFO:tensorflow:loss = 1204.1074, step = 15901 (1.049 sec)
INFO:tensorflow:global_step/sec: 98.7688
INFO:tensorflow:loss = 4091.7756, step = 16001 (1.013 sec)
INFO:tensorflow:global_step/sec: 102.807
INFO:tensorflow:loss = 1079.3315, step = 16101 (0.973 sec)
INFO:tensorflow:global_step/sec: 101.543
INFO:tensorflow:loss = 1063.1769, step = 16201 (0.985 sec)
INFO:tensorflow:global_step/sec: 102.83
INFO:tensorflow:loss = 1755.646, step = 16301 (0.972 sec)
INFO

INFO:tensorflow:loss = 1476.3496, step = 23601 (1.084 sec)
INFO:tensorflow:global_step/sec: 106.32
INFO:tensorflow:loss = 1314.6456, step = 23701 (0.940 sec)
INFO:tensorflow:global_step/sec: 101.279
INFO:tensorflow:loss = 1310.6016, step = 23801 (0.987 sec)
INFO:tensorflow:global_step/sec: 104.084
INFO:tensorflow:loss = 1414.846, step = 23901 (0.961 sec)
INFO:tensorflow:global_step/sec: 89.1024
INFO:tensorflow:loss = 2265.8713, step = 24001 (1.122 sec)
INFO:tensorflow:global_step/sec: 86.5018
INFO:tensorflow:loss = 36029.008, step = 24101 (1.157 sec)
INFO:tensorflow:global_step/sec: 83.6548
INFO:tensorflow:loss = 1122.5024, step = 24201 (1.194 sec)
INFO:tensorflow:global_step/sec: 95.978
INFO:tensorflow:loss = 921.52124, step = 24301 (1.042 sec)
INFO:tensorflow:global_step/sec: 95.0524
INFO:tensorflow:loss = 1148.5303, step = 24401 (1.052 sec)
INFO:tensorflow:global_step/sec: 103.311
INFO:tensorflow:loss = 1241.2742, step = 24501 (0.968 sec)
INFO:tensorflow:global_step/sec: 98.952
INFO

INFO:tensorflow:global_step/sec: 103.621
INFO:tensorflow:loss = 1176.645, step = 31901 (0.965 sec)
INFO:tensorflow:global_step/sec: 102.779
INFO:tensorflow:loss = 1314.2837, step = 32001 (0.973 sec)
INFO:tensorflow:global_step/sec: 104.319
INFO:tensorflow:loss = 1056.4753, step = 32101 (0.959 sec)
INFO:tensorflow:global_step/sec: 101.446
INFO:tensorflow:loss = 1358.4912, step = 32201 (0.986 sec)
INFO:tensorflow:global_step/sec: 102.01
INFO:tensorflow:loss = 1328.8494, step = 32301 (0.980 sec)
INFO:tensorflow:global_step/sec: 103.172
INFO:tensorflow:loss = 4979.5127, step = 32401 (0.969 sec)
INFO:tensorflow:global_step/sec: 103.459
INFO:tensorflow:loss = 1149.0238, step = 32501 (0.967 sec)
INFO:tensorflow:global_step/sec: 104.777
INFO:tensorflow:loss = 1240.0249, step = 32601 (0.955 sec)
INFO:tensorflow:global_step/sec: 103.349
INFO:tensorflow:loss = 1185.6511, step = 32701 (0.968 sec)
INFO:tensorflow:global_step/sec: 102.814
INFO:tensorflow:loss = 949.0501, step = 32801 (0.972 sec)
INF

...

INFO:tensorflow:global_step/sec: 102.835
INFO:tensorflow:loss = 1601.7712, step = 48401 (0.972 sec)
INFO:tensorflow:global_step/sec: 102.356
INFO:tensorflow:loss = 2209.832, step = 48501 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.817
INFO:tensorflow:loss = 1499.8091, step = 48601 (0.973 sec)
INFO:tensorflow:global_step/sec: 102.721
INFO:tensorflow:loss = 911.5684, step = 48701 (0.973 sec)
INFO:tensorflow:global_step/sec: 103.487
INFO:tensorflow:loss = 587.26526, step = 48801 (0.967 sec)
INFO:tensorflow:global_step/sec: 104.315
INFO:tensorflow:loss = 722.183, step = 48901 (0.958 sec)
INFO:tensorflow:global_step/sec: 104.868
INFO:tensorflow:loss = 600.838, step = 49001 (0.954 sec)
INFO:tensorflow:global_step/sec: 100.812
INFO:tensorflow:loss = 695.6111, step = 49101 (0.992 sec)
INFO:tensorflow:global_step/sec: 103.307
INFO:tensorflow:loss = 923.04895, step = 49201 (0.968 sec)
INFO:tensorflow:global_step/sec: 103.844
INFO:tensorflow:loss = 1171.4175, step = 49301 (0.963 sec)
INFO:te

INFO:tensorflow:global_step/sec: 101.999
INFO:tensorflow:loss = 488.4104, step = 56701 (0.980 sec)
INFO:tensorflow:global_step/sec: 102.315
INFO:tensorflow:loss = 483.06427, step = 56801 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.883
INFO:tensorflow:loss = 703.1721, step = 56901 (0.972 sec)
INFO:tensorflow:global_step/sec: 103.524
INFO:tensorflow:loss = 631.0388, step = 57001 (0.966 sec)
INFO:tensorflow:global_step/sec: 102.376
INFO:tensorflow:loss = 467.21625, step = 57101 (0.977 sec)
INFO:tensorflow:global_step/sec: 103.148
INFO:tensorflow:loss = 578.8094, step = 57201 (0.969 sec)
INFO:tensorflow:global_step/sec: 103.391
INFO:tensorflow:loss = 4821.3965, step = 57301 (0.967 sec)
INFO:tensorflow:global_step/sec: 103.105
INFO:tensorflow:loss = 714.5042, step = 57401 (0.970 sec)
INFO:tensorflow:global_step/sec: 103.739
INFO:tensorflow:loss = 501.97144, step = 57501 (0.964 sec)
INFO:tensorflow:global_step/sec: 102.802
INFO:tensorflow:loss = 960.61646, step = 57601 (0.973 sec)
INFO:

INFO:tensorflow:loss = 511.75195, step = 62901 (1.024 sec)
INFO:tensorflow:global_step/sec: 96.4211
INFO:tensorflow:loss = 498.7827, step = 63001 (1.037 sec)
INFO:tensorflow:global_step/sec: 95.7323
INFO:tensorflow:loss = 4823.0137, step = 63101 (1.045 sec)
INFO:tensorflow:global_step/sec: 102.272
INFO:tensorflow:loss = 435.76184, step = 63201 (0.977 sec)
INFO:tensorflow:global_step/sec: 103.208
INFO:tensorflow:loss = 704.23987, step = 63301 (0.969 sec)
INFO:tensorflow:global_step/sec: 102.371
INFO:tensorflow:loss = 489.52008, step = 63401 (0.977 sec)
INFO:tensorflow:global_step/sec: 99.5213
INFO:tensorflow:loss = 10045.898, step = 63501 (1.005 sec)
INFO:tensorflow:global_step/sec: 103.979
INFO:tensorflow:loss = 2999.3262, step = 63601 (0.962 sec)
INFO:tensorflow:global_step/sec: 103.203
INFO:tensorflow:loss = 650.026, step = 63701 (0.969 sec)
INFO:tensorflow:global_step/sec: 104.037
INFO:tensorflow:loss = 1073.2104, step = 63801 (0.961 sec)
INFO:tensorflow:global_step/sec: 100.933
INF

INFO:tensorflow:global_step/sec: 102.263
INFO:tensorflow:loss = 853.1785, step = 71201 (0.978 sec)
INFO:tensorflow:global_step/sec: 102.752
INFO:tensorflow:loss = 2808.7146, step = 71301 (0.973 sec)
INFO:tensorflow:global_step/sec: 103.135
INFO:tensorflow:loss = 459.74365, step = 71401 (0.970 sec)
INFO:tensorflow:global_step/sec: 103.648
INFO:tensorflow:loss = 834.5559, step = 71501 (0.965 sec)
INFO:tensorflow:global_step/sec: 102.634
INFO:tensorflow:loss = 574.97015, step = 71601 (0.974 sec)
INFO:tensorflow:global_step/sec: 102.676
INFO:tensorflow:loss = 740.5486, step = 71701 (0.974 sec)
INFO:tensorflow:global_step/sec: 102.407
INFO:tensorflow:loss = 386.47372, step = 71801 (0.976 sec)
INFO:tensorflow:global_step/sec: 102.027
INFO:tensorflow:loss = 702.3493, step = 71901 (0.980 sec)
INFO:tensorflow:global_step/sec: 103.703
INFO:tensorflow:loss = 407.55316, step = 72001 (0.964 sec)
INFO:tensorflow:global_step/sec: 103.208
INFO:tensorflow:loss = 389.61453, step = 72101 (0.969 sec)
INFO

INFO:tensorflow:global_step/sec: 102.334
INFO:tensorflow:loss = 495.3766, step = 79501 (0.977 sec)
INFO:tensorflow:global_step/sec: 91.6443
INFO:tensorflow:loss = 448.673, step = 79601 (1.091 sec)
INFO:tensorflow:global_step/sec: 102.041
INFO:tensorflow:loss = 491.87885, step = 79701 (0.980 sec)
INFO:tensorflow:global_step/sec: 100.738
INFO:tensorflow:loss = 610.23865, step = 79801 (0.993 sec)
INFO:tensorflow:global_step/sec: 100.786
INFO:tensorflow:loss = 751.46533, step = 79901 (0.992 sec)
INFO:tensorflow:Saving checkpoints for 80000 into ./ouput_cylinder_dnn/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-27-01:32:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ouput_cylinder_dnn/model.ckpt-80000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-27-01:32:51
INFO:tensorflow:Saving dict f